In [2]:
import dxdata
import pandas as pd
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

In [3]:
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)

In [4]:
data = dataset['participant']

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("YourAppName")\
    .config("spark.kryoserializer.buffer.mb", "24")\
    .config("spark.kryoserializer.buffer.max.mb", "2000")\
    .getOrCreate()



# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

In [4]:
field_list = [field_eid, field_height, field_sex, field_age]
field_list

[<Field "eid">, <Field "p50_i0">, <Field "p31">, <Field "p21022">]

## prepare recorders 

In [16]:
df_all_fields = pd.DataFrame(columns = ['entity','field_name','filed_title'])
for entity in dataset.entities:
    entity_name  = entity.name
    
    for field in entity.fields:
        field_name = field.name
        filed_title = field.title
        df_all_fields.loc[len(df_all_fields),] = [entity_name,field_name,filed_title]
df_all_fields['downloaded']=[False]*len(df_all_fields)
df_all_fields.to_csv('recorder_all_fields.csv',index=False)


In [24]:
for entity in df_all_fields['entity'].unique():
    df_target_entity = df_all_fields.loc[df_all_fields['entity']==entity,]
    df_target_entity.to_csv(f'recorder_{entity}.csv')

# data downloading codes 

In [6]:
entity='participant'
ind = 18
interval = 21
bulk_num = 30


In [ ]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')
end = ind+interval
ind
while ind < end:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_lst = [data.find_field(x) for x in list(field_names)]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    except:
        print(f"error in bulk {ind}")
        fail_list+=[ind]
    ind+=1
print(f' starttime {start_time}, endtime {datetime.now()}')

In [14]:
fail_list = [52]

In [15]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')

for ind in fail_list:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_lst = [data.find_field(x) for x in list(field_names)]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
        fail_list.remove(ind)
    except Exception as e:
        print(e)
        print(f"error in bulk {ind}")
        
    ind+=1
print(f' starttime {start_time}, endtime {datetime.now()}')

now download bulk 816, from 24481 to 24510
 starttime 2024-02-27 09:46:23.682985, endtime 2024-02-27 09:46:48.930128


In [ ]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')

ind = 52
try:
    print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
    field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
    index = field_names.index
    field_lst = [data.find_field(x) for x in list(field_names)]

    df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
    df_download = df_download.toPandas()

    df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

    df_target_entity.loc[index, 'downloaded']=[True]*len(index)
    df_target_entity.loc[index, 'ind']=[ind]*len(index)
    df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    fail_list.remove(ind)
except Exception as e:
    print(e)
    print(f"error in bulk {ind}")


print(f' starttime {start_time}, endtime {datetime.now()}')

ind = 0
interval = 5

filenames = [f'data/{entity}/{x}.csv' for x in range(ind,ind+interval)]
df_compact = pd.concat(map(pd.read_csv, filenames))
df_compact.to_pickle(f'data/{entity}/{ind}-{interval-1}.pkl')
for file in filenames:
    os.remove(file)
